In [1]:
from premise import *
from datapackage import Package
import bw2io as bw

In [2]:
import bw2data
bw2data.projects.set_current("ei39")

In [3]:
fp = r"/Users/romain/GitHub/RTE_scenarios/datapackage.json"
FE2050 = Package(fp)

In [4]:
ndb = NewDatabase(
    scenarios=[
        #{"model":"remind", "pathway":"SSP2-Base", "year":2050},
        #{"model":"remind", "pathway":"SSP2-Base", "year":2010},
        #{"model":"image", "pathway":"SSP2-Base", "year":2020},
        {"model":"image", "pathway":"SSP2-Base", "year":2050},
        {"model":"image", "pathway":"SSP2-Base", "year":2040},
        #{"model":"image", "pathway":"SSP2-Base", "year":2050},
        #{"model":"image", "pathway":"SSP2-Base", "year":2050},
    ],
    source_db="ecoinvent 3.9.1 cutoff", # <-- name of the database in the BW2 project. Must be a string.
    source_version="3.9", # <-- version of ecoinvent. Can be "3.5", "3.6", "3.7" or "3.8". Must be a string.
    key="tUePmX_S5B8ieZkkM7WUU2CnO8SmShwmAeWK9x2rTFo=",
    #external_scenarios=[FE2050,]
    use_multiprocessing=True,
    #system_model="consequential"
    # to be requested from the library maintainers if you want to use default scenarios included in `premise`
)

premise v.(2, 1, 0, 'dev0')
+------------------------------------------------------------------+
| Warning                                                          |
+------------------------------------------------------------------+
| Because some of the scenarios can yield LCI databases            |
| containing net negative emission technologies (NET),             |
| it is advised to account for biogenic CO2 flows when calculating |
| Global Warming potential indicators.                             |
| `premise_gwp` provides characterization factors for such flows.  |
| It also provides factors for hydrogen emissions to air.          |
|                                                                  |
| Within your bw2 project:                                         |
| from premise_gwp import add_premise_gwp                          |
| add_premise_gwp()                                                |
+------------------------------------------------------------------+
+-----

In [5]:
#ndb.update_electricity()
#ndb.update_all()
ndb.update()

`update()` will skip the following sectors: 'buses', 'cars', 'two_wheelers'.
If you want to update these sectors, please run them separately afterwards.


Updating: external: 100%|█████████████| 10/10 [06:37<00:00, 39.73s/it]


Done!



In [6]:
ndb.write_db_to_matrices()

Write new database(s) to matrix.
Generate scenario report.
Report saved under /Users/romain/Github/premise/dev/export/scenario_report.
Generate change report.
Report saved under /Users/romain/Github/premise/dev.


In [9]:
from multiprocessing import Process, current_process
import logging
import os

def configure_logging(process_id):
    """Configures logging for a process with a unique file per process."""
    log_filename = f"process_{process_id}.log"
    
    logger = logging.getLogger(f'worker_{process_id}')
    logger.setLevel(logging.DEBUG)
    file_handler = logging.FileHandler(log_filename, mode='a')
    formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
    file_handler.setFormatter(formatter)
    logger.addHandler(file_handler)

    logger.propagate = False
    return logger

def main():
    processes = []
    for _ in range(5):
        # Start each process with a unique configuration based on the process ID.
        process = Process(target=lambda: configure_logging(current_process().pid).info(f"This is an info message from process {current_process().pid}."))
        processes.append(process)
        process.start()

    for process in processes:
        process.join()

if __name__ == "__main__":
    main()


AttributeError: Can't pickle local object 'main.<locals>.<lambda>'

In [ ]:
import bw2data
bw2data.projects.set_current("new4")

In [ ]:
preserve = [
    "biosphere3",
    "ecoinvent 3.9.1 cutoff",
    "ecoinvent 3.9.1 consequential",
    "ecoinvent EN15804 3.9.1",
    "ecoinvent 3.6 cutoff",
    "ecoinvent 3.7 cutoff",
    "ecoinvent 3.8 cutoff",
    
]

for db in list(bw2data.databases):
    if db not in preserve:
        del bw2data.databases[db]

In [ ]:
ndb.write_db_to_brightway()

In [ ]:
import numpy as np
import pandas as pd
df = pd.DataFrame(
np.hstack(
    [ndb.scenarios[0]["iam data"].electricity_markets.sel(region="EUR"),
    ndb.scenarios[1]["iam data"].electricity_markets.sel(region="EUR"),
    ndb.scenarios[2]["iam data"].electricity_markets.sel(region="EUR"),
    ndb.scenarios[3]["iam data"].electricity_markets.sel(region="EUR"),]
), columns=[2020, 2030, 2040, 2050],)

In [ ]:
df.index = ['Geothermal', 'Biomass CHP', 'Gas CHP', 'Wind Offshore', 'Coal PC CCS',
 'Coal CHP','Gas CC CCS','Gas ST','Oil ST','Coal IGCC','Gas CC','Coal PC',
 'Biomass IGCC CCS','Hydro','Storage, Hydrogen','Coal IGCC CCS','Nuclear',
 'Wind Onshore','Biomass IGCC','Solar PV Centralized','Solar CSP']

In [ ]:
df

In [ ]:
ndb.generate_scenario_report()

In [ ]:
ndb.

In [ ]:
import json
import os
import glob
from pprint import pprint

#fp="/Users/romain/Documents/premise_test/flows"
fp="/Users/romain/Documents/test_olca_39_4/flows"

l = []
#Load the images from images folder.
for f in glob.glob(f'{fp}/*.json'):
    with open(f) as fd:
     json_data = json.load(fd)
     l.append([json_data["name"], json_data["category"], json_data["@id"]])
    

In [ ]:
import pandas as pd
pd.DataFrame(l, columns=["Source flow", "Source category", "id"]).to_excel("mapping_source_2.xlsx")

In [ ]:
import csv


fp_in = "/Users/romain/Documents/flow_mapping_premise4_.csv"
fp_out = "/Users/romain/Documents/flow_mapping_premise5.csv"

l = []
with open(fp_in) as csv_file:
    csv_reader = csv.reader(csv_file, delimiter=',')
    for r in csv_reader:
        l.append(r)
    
with open(fp_out, 'w', newline='') as file:
    writer = csv.writer(file, delimiter=';')
    writer.writerows(l)

In [ ]:
import csv


fp_in = "/Users/romain/Downloads/sp_flow_import_map.csv"
fp_out = "/Users/romain/Downloads/sp_flow_import_map2.csv"

l = []
with open(fp_in) as csv_file:
    csv_reader = csv.reader(csv_file, delimiter=',')
    for r in csv_reader:
        l.append(r)
    
with open(fp_out, 'w', newline='') as file:
    writer = csv.writer(file, delimiter=';')
    writer.writerows(l)

In [ ]:
ndb.update_all()

In [ ]:
scenarios = [
    {"model": "remind", "pathway":"SSP1-Base", "year": 2050},
    {"model": "remind", "pathway":"SSP1-NDC", "year": 2050},
    #{"model": "remind", "pathway":"SSP1-NPi", "year": 2050},
    #{"model": "remind", "pathway":"SSP1-PkBudg1150", "year": 2050},
    #{"model": "remind", "pathway":"SSP1-PkBudg500", "year": 2050},
    #{"model": "remind", "pathway":"SSP5-Base", "year": 2050},
    #{"model": "remind", "pathway":"SSP5-NDC", "year": 2050},
    #{"model": "remind", "pathway":"SSP5-NPi", "year": 2050},
    #{"model": "remind", "pathway":"SSP5-PkBudg1150", "year": 2050},
    #{"model": "remind", "pathway":"SSP5-PkBudg500", "year": 2050},
]

ndb = NewDatabase(
        scenarios = scenarios,        
        source_db="ecoinvent 3.9 cutoff",
        source_version="3.9",
        key='tUePmX_S5B8ieZkkM7WUU2CnO8SmShwmAeWK9x2rTFo=',
)


In [ ]:
ndb.update_all()

In [ ]:
ndb.write_superstructure_db_to_brightway("test")

In [ ]:
bw.BW2Package.export_obj(bw.Database("test"))

In [ ]:
import numpy as np
for dataset in bw.Database("test"):
    #for exc in dataset.exchanges():
    #    if not isinstance(exc["amount"], float):
    #        print(dataset["name"], type(exc["amount"]))

    #    if isinstance(exc["amount"], np.float64):
    #        print(dataset["name"], type(exc["amount"]))

    for k, v in dataset.items():
        if isinstance(v, dict):
            for i, j in v.items():
                if isinstance(j, np.float64):
                    #continue
                    print(i, j)
                    
                if not type(j) in [str, float, int, tuple, list, dict, bool, None]:
                    print("1", dataset["name"], i, j, type(j), float(j))

    for e in dataset.exchanges():
        for k, v in e.items():
            if isinstance(v, np.float64):
                #continue
                print(k, v)
                
            if not type(v) in [str, float, int, tuple, list, dict, bool, None]:
                print("2", dataset["name"], k, v, type(v), float(v))

In [ ]:
ndb.write_datapackage()

In [ ]:
from unfold import Unfold
import brightway2 as bw
bw.projects.set_current("ei39")

In [ ]:
u = Unfold("export/datapackage/datapackage_2023-04-20.zip")

In [ ]:
u.unfold(superstructure=True, name="my_super_db")

In [ ]:
dict(enumerate(["a", "b", "c"]))

In [ ]:
ndb.write_db_to_brightway("test")

In [ ]:
variables = ndb.scenarios[0]["iam data"].fuel_markets.variables.values
ndb.scenarios[0]["iam data"].fuel_markets.sel(region="EUR", variables=[v for v in variables if "diesel" in v], year=2050).to_dataframe("val")

In [ ]:
ndb.scenarios[0]["iam data"].gains_data_EU

In [ ]:
ndb.write_db_to_brightway("test")

In [ ]:
ndb.write_datapackage()

In [ ]:
from unfold import Unfold

In [ ]:
u = Unfold("/Users/romain/Downloads/premise_remind_ssp2_base.zip")

In [ ]:
u.unfold(scenarios=[98, 99])

In [ ]:
from unfold import Fold

In [ ]:
f = Fold()

In [ ]:
f.fold()

In [ ]:
import premise

In [ ]:
premise.__file__

In [ ]:
from premise import clear_cache

In [ ]:
clear_cache()